In [1]:
import data
import model
from random import choice, seed
import numpy as np

Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [2]:
np.random.seed(1337)  # TODO repeatability
seed(1337)

In [3]:
X, y, _, _, _ = data.load("~/Documents/flavours-of-physics-start/input/training.csv",
                          shuffle=True)
Xa, ya, wa, _, _ = data.load("~/Documents/flavours-of-physics-start/input/check_agreement.csv",
                             shuffle=False, weight=True)
Xc, _, _, mc, _ = data.load("~/Documents/flavours-of-physics-start/input/check_correlation.csv",
                            shuffle=False, mass=True, test=True)

X, scaler = data.preprocess_data(X)
Xa, _ = data.preprocess_data(Xa)
Xc, _ = data.preprocess_data(Xc)

X_test, _, _, _, ids = data.load("~/Documents/flavours-of-physics-start/input/test.csv", test=True,
                                 ids=True)
X_test = scaler.transform(X_test)
print "Done"

In [ ]:
n_models = 1
n_epochs = 30
probs = None
for i in range(n_models):
    # l = choice(xrange(1, 50))
    l = 0
    print "Model {}; l = {}".format(i, l)
    m = model.create_model(X.shape[1], l)
    # model.fit_model_channels(m, X, y, Xa, ya, wa, Xc, mc)
    model.fit_model_mc(m, X, y, Xa, ya, wa, Xc, mc,
                       epoch_count=n_epochs, batch_size=64, validation_split=0.1)
    p = model.predict_model(m, np.array(X_test))
    probs = p if probs is None else p + probs
probs /= n_models

In [ ]:
random_classifier = np.random.rand(len(probs))
q = 0.98
combined_probs = q * (probs ** 30) + (1 - q) * random_classifier

In [ ]:
data.save_submission(ids, combined_probs, "grl_prediction.csv")